##### <span style="color:#e74c3c;">🔧 1. Configuración e Importación de Librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import levene, shapiro, f_oneway, f, kruskal, rankdata, chi2, chi2_contingency
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('./Diabetes_and_LifeStyle_Dataset .csv')

### <span style="color:#3498db;"> ¿Existen diferencias significativas en el riesgo promedio de diabetes entre los tres grupo de géneros: hombre, mujer y otro?

**Objetivo del análisis**

El objetivo de este análisis es determinar si existen diferencias estadísticamente significativas en el riesgo de diabetes entre los distintos grupos de género presentes en la muestra (hombre, mujer y otro). Identificar posibles diferencias asociadas al género resulta relevante para orientar estrategias de prevención, detección temprana y diseño de políticas de salud pública enfocadas en poblaciones específicas. Para ello utilizaremos el m[etodo de Kruskal-Wallis.]

**Justificación del uso de la prueba de Kruskal–Wallis**

La prueba de Kruskal–Wallis es un método no paramétrico utilizado para comparar grupos independientes cuando no se cumplen los supuestos de normalidad o homogeneidad de varianzas requeridos por pruebas paramétricas como el ANOVA.

En este estudio, el uso de esta prueba está justificado por las siguientes razones:

1. La variable dependiente, riesgo de diabetes (diabetes_risk_score), es una medida continua que puede presentar asimetría o valores atípicos.

2. La variable independiente, género, es categórica con tres grupos independientes.

3. No existe una distribución normal del riesgo de diabetes dentro de cada grupo.

4. La prueba de Kruskal–Wallis es robusta frente a distribuciones no normales y tamaños de muestra desiguales.

Por estas razones, esta prueba constituye una herramienta estadística adecuada y confiable para evaluar diferencias entre los grupos de género en este contexto.

**Hipótesis estadísticas**

Hipótesis nula (H₀): Las distribuciones del riesgo de diabetes son iguales para los tres grupos de géneros.

Hipótesis alternativa (H₁): Al menos uno de los grupos presenta una distribución diferente.

Comprobacion de que no se cumple la normalidad utilizando el metodo Shapiro-Wilk

In [3]:
# Grupos
groups = ['Male', 'Female', 'Other']

# Revisar normalidad con Shapiro-Wilk
normality_results = {}

for g in groups:
    data = df[df['gender'] == g]['diabetes_risk_score'].dropna()
    stat, p = shapiro(data)
    normality_results[g] = (stat, p)

for g, (stat, p) in normality_results.items():
    if p < 0.05:
        print(f"El grupo {g} no cumple normalidad")
    else:
        print(f"El grupo {g} cumple normalidad")

El grupo Male no cumple normalidad
El grupo Female no cumple normalidad
El grupo Other no cumple normalidad


In [ ]:
# Definición de los grupos independientes
data_groups = [df[df['gender'] == g]['diabetes_risk_score'].dropna() for g in groups]

for g, data in zip(groups, data_groups):
    print(f"{g}: n = {len(data)}")

# Combinación de todas las observaciones
all_data = np.concatenate(data_groups)
# Asignación de rangos
ranks = rankdata(all_data) 

# Redistribución de rangos por grupo
start = 0
ranked_groups = []
for i, data in enumerate(data_groups):
    n = len(data)
    ranked_groups.append(ranks[start:start+n])
    start += n

for g, r in zip(groups, ranked_groups):
    print(f"Suma de rangos para {g}:", np.sum(r))


#calcular estadístico H de Kruskal-Wallis
# Fórmula:
#
#            12
# H = ----------------- * Σ (R_i² / n_i) − 3(N + 1)
#        N (N + 1)
# donde:
# - k   = número de grupos
# - n_i = tamaño del grupo i
# - R_i = suma de rangos del grupo i
# - N   = tamaño total de la muestra
#
# Bajo la hipótesis nula, H sigue aproximadamente una
# distribución chi-cuadrado con (k − 1) grados de libertad.

print("\nCálculo del estadístico H de Kruskal-Wallis:")
k = len(groups)
n_i = [len(g) for g in data_groups]
N = sum(n_i)
R_i = [np.sum(r) for r in ranked_groups]

H = (12/(N*(N+1))) * sum(R_i[i]**2 / n_i[i] for i in range(k)) - 3*(N+1)
print(f"\nEstadístico H = {H:.4f}")


Male: n = 46460
Female: n = 48873
Other: n = 1964
Suma de rangos para Male: 2261529067.0
Suma de rangos para Female: 2376645876.5
Suma de rangos para Other: 95226809.5

Cálculo del estadístico H de Kruskal-Wallis:

Estadístico H = 0.1366


Determinacion de region de confianza:

Distribución de referencia: χ² con k-1 grados de libertad.

Grados de libertad: df = k - 1 = 2

Nivel de significancia: α = 0.05

In [17]:
df_ = k-1
chi2_crit = chi2.ppf(0.95, df_)
print(f"Región de rechazo: H > {chi2_crit:.4f}")
print("Si H > χ² crítico → Rechazamos H₀.")
print("Si H ≤ χ² crítico → No rechazamos H₀.")

Región de rechazo: H > 5.9915
Si H > χ² crítico → Rechazamos H₀.
Si H ≤ χ² crítico → No rechazamos H₀.


Dado que el valor del estadístico H es considerablemente menor que el valor crítico de la distribución chi-cuadrado, no se rechaza la hipótesis nula.
##### Conclusión del análisis

Los resultados del análisis indican que no existen diferencias estadísticamente significativas en el riesgo de diabetes entre los distintos grupos de género en la muestra analizada, al nivel de significancia del 5%. Esto sugiere que, dentro de este conjunto de datos, el género no parece ser un factor determinante en la variación del riesgo promedio de diabetes.

### <span style="color:#3498db;"> ¿El tabaquismo influye en la etapa de la diabetes?

**Objetivo del análisis:** 

El objetivo de esta prueba es evaluar si existe una asociación estadísticamente significativa entre la etapa de la diabetes y el estado de tabaquismo. Determinar esta relación es fundamental para comprender el impacto del tabaquismo en la progresión de la diabetes y para orientar intervenciones preventivas y clínicas.

**Justificación del uso de la prueba Chi-cuadrado de independencia:**

La prueba Chi-cuadrado de independencia se utiliza para evaluar si existe una relación estadísticamente significativa entre dos variables categóricas. En el presente estudio, esta prueba resulta metodológicamente adecuada debido a las características de las variables analizadas y al objetivo planteado.

En primer lugar, ambas variables de interés (la etapa de la diabetes y el estado de tabaquismo) son variables cualitativas categóricas, lo cual cumple con el requisito fundamental para la aplicación de esta prueba. La etapa de la diabetes clasifica a los individuos según el grado de progresión de la enfermedad, mientras que el tabaquismo categoriza el hábito de consumo de tabaco.

En segundo lugar, el objetivo del análisis no es comparar medias ni distribuciones, sino evaluar la existencia de una asociación o dependencia entre las dos variables. La prueba Chi-cuadrado permite contrastar las frecuencias observadas con las frecuencias esperadas bajo el supuesto de independencia, lo que la convierte en una herramienta adecuada para responder a esta pregunta de investigación.

Asimismo, la prueba asume la independencia de las observaciones, condición que se cumple al tratarse de registros individuales correspondientes a distintos sujetos. Además, se verifica el cumplimiento del supuesto de frecuencias esperadas mínimas, garantizando la validez de la aproximación de la distribución chi-cuadrado.

**Hipótesis Estadistica**

Hipótesis nula (H₀): Las variables son independientes (no hay asociación)

Hipótesis alternativa (H₁): Hay asociación entre las variables


In [ ]:
# Tabla de contingencia: diabetes_stage vs smoking_status
# La tabla de contingencia resume el número de individuos
# en cada combinación de categorías:
# - Filas: etapa de la diabetes
# - Columnas: estado de tabaquismo

contingency_table = pd.crosstab(df['diabetes_stage'], df['smoking_status'])
print(contingency_table)

#  Cálculo del estadístico Chi-cuadrado
# La función chi2_contingency() calcula:
# - χ²: estadístico de prueba
# - p-value: probabilidad asociada
# - dof: grados de libertad
# - expected: frecuencias esperadas bajo H₀
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
print(f"\nChi2 = {chi2_stat:.3f}, p = {p_val:.4f}, dof = {dof}")

# Conclusión
if p_val < 0.05:
    print("Conclusión: Hay asociación significativa entre las variables")
else:
    print("Conclusión: No hay evidencia de asociación significativa")

smoking_status  Current  Former  Never
diabetes_stage                        
Gestational          57      49    161
No Diabetes        1574    1546   4617
Pre-Diabetes       6237    6191  18585
Type 1               19      25     73
Type 2            11704   11677  34782

Chi2 = 2.105, p = 0.9776, dof = 8
Conclusión: No hay evidencia de asociación significativa


**Conclusión del análisis**
Con base en los resultados de la prueba Chi-cuadrado de independencia, no se encontró evidencia estadísticamente significativa de asociación entre la etapa de la diabetes y el estado de tabaquismo, al nivel de significancia α = 0.05. En consecuencia, no se rechaza la hipótesis nula, lo que indica que, en la muestra analizada, ambas variables se comportan de manera independiente.

### <span style="color:#3498db;"> ¿Existen diferencias significativas en el índice de masa corporal promedio entre las distintas etapas de la diabetes?

**Objetivo del Análisis:**
Este análisis busca determinar si el IMC varía sistemáticamente a medida que cambia la etapa de la diabetes, lo que permitiría identificar si el peso corporal constituye un factor diferenciador en la progresión de la enfermedad. Los resultados obtenidos aportan evidencia empírica relevante para comprender el papel del IMC en el contexto de la diabetes y para orientar decisiones clínicas y estrategias de prevención basadas en el control del peso y otros factores de riesgo metabólico.

**Justificación del uso de la prueba ANOVA:**

La prueba ANOVA de una vía se utiliza para comparar las medias de una variable continua entre tres o más grupos independientes definidos por una variable categórica.

En este estudio, el uso de ANOVA está justificado porque:

1. La variable dependiente, índice de masa corporal (BMI), es cuantitativa continua.

2. La variable independiente, etapa de diabetes (diabetes_stage), es categórica con múltiples niveles.

3. El objetivo del análisis es comparar medias, no asociaciones ni distribuciones.

4. Se evalúa el cumplimiento de los supuestos de normalidad y homogeneidad de varianzas, garantizando la validez del método.

Por estas razones, el ANOVA de una vía es el procedimiento estadístico más adecuado para responder a la pregunta planteada.

**Hipótesis Estadística**

H₀: Las medias de IMC son iguales entre todas las etapas de diabetes.

H₁: Al menos una etapa tiene un IMC promedio diferente.


##### Verificación de los supuestos:

In [7]:
levels = df["diabetes_stage"].dropna().unique()
data_groups = [df[df["diabetes_stage"] == level]['bmi'].dropna() for level in levels]
    
    
print("Normalidad (Shapiro-Wilk) por grupo:")
normality_results = []
for level, data in zip(levels, data_groups):
    if len(data) > 200:
        data_sample = data.sample(200, random_state=42)
    else:
        data_sample = data
    stat, p = shapiro(data_sample)
    result = 'Normal' if p >= 0.05 else 'No normal'
    normality_results.append(result)
    print(f"  {level}: W={stat:.3f}, p={p:.4f} -> {result}")

# Homogeneidad de varianzas (Levene)
stat_levene, p_levene = levene(*data_groups)
hom_var_result = 'Varianzas homogéneas' if p_levene >= 0.05 else 'Varianzas diferentes'

print(f"\nHomogeneidad de varianzas (Levene): W={stat_levene:.3f}, p={p_levene:.4f} -> {hom_var_result}")


Normalidad (Shapiro-Wilk) por grupo:
  Type 2: W=0.996, p=0.9035 -> Normal
  No Diabetes: W=0.991, p=0.2120 -> Normal
  Pre-Diabetes: W=0.994, p=0.6584 -> Normal
  Gestational: W=0.992, p=0.3020 -> Normal
  Type 1: W=0.986, p=0.2541 -> Normal

Homogeneidad de varianzas (Levene): W=1.005, p=0.4035 -> Varianzas homogéneas


In [9]:
#  ANOVA
f_stat, p_val = f_oneway(*data_groups)

#  Región de rechazo
k = len(levels)        # numero de grupos
N = sum([len(g) for g in data_groups])  # tamano total de la muestra
df1 = k - 1
df2 = N - k
F_crit = f.ppf(0.95, df1, df2)

# Conclusión
conclusion = "Rechazamos H₀ → Al menos un grupo tiene media diferente" if p_val < 0.05 \
                else "No se rechaza H₀ → No hay diferencias significativas entre grupos"

# Resumen descriptivo por grupo
summary = []
for level, data in zip(levels, data_groups):
    summary.append({
        'diabetes_stage': level,
        'n': len(data),
        'mean': data.mean(),
        'std': data.std(),
        'median': data.median(),
        'min': data.min(),
        'max': data.max()
    })
summary_df = pd.DataFrame(summary)

print(f"F = {f_stat:.3f}, p = {p_val:.4f}")
print(f"Región de rechazo: F > {F_crit:.3f}")
print(f"Conclusión: {conclusion}")
print("\nResumen descriptivo por grupo:")
print(summary_df)


F = 295.576, p = 0.0000
Región de rechazo: F > 2.372
Conclusión: Rechazamos H₀ → Al menos un grupo tiene media diferente

Resumen descriptivo por grupo:
  diabetes_stage      n       mean       std  median   min   max
0         Type 2  58163  25.902251  3.579610    25.9  15.0  39.2
1    No Diabetes   7737  24.627065  3.550143    24.6  15.0  38.4
2   Pre-Diabetes  31013  25.331867  3.545686    25.3  15.0  39.0
3    Gestational    267  25.007116  3.652477    24.8  17.0  36.1
4         Type 1    117  24.661538  3.711767    24.6  17.4  33.5


**Conclusiones**

Los resultados del análisis de varianza (ANOVA) indican que existen diferencias estadísticamente significativas en el índice de masa corporal promedio entre las distintas etapas de la diabetes (F = 295.576, p < 0.0001). Esto significa que al menos una etapa de diabetes presenta un IMC promedio diferente de las demás.

### <span style="color:#3498db;"> ¿Existe una diferencia estadísticamente significativa en la presión arterial sistólica entre individuos diagnosticados con diabetes y aquellos sin diagnóstico?

**Objetivo del Análisis:**

Este análisis permite determinar si la condición de diabetes se asocia con cambios en la presión sistólica, lo cual es relevante para identificar riesgos cardiovasculares y orientar estrategias de prevención y manejo clínico en la población.

**Justificación del uso de la prueba t**

La prueba t de Student para muestras independientes es el método adecuado cuando:

1. La variable dependiente es continua (systolic_bp).

2. La variable independiente es binaria o dicotómica (diagnosed_diabetes: 0 = no, 1 = sí).

3. El objetivo es comparar medias entre dos grupos independientes.

Se verifica además el cumplimiento de los supuestos de normalidad y homogeneidad de varianzas para garantizar la validez de la prueba.

**Hipótesis Estadística**

H₀ (nula): La presión sistólica media es igual en personas con y sin diagnóstico de diabetes.

H₁ (alternativa): La presión sistólica media es diferente entre ambos grupos.


##### Verificación de supuestos

In [11]:

alpha = 0.05

g0 = df.loc[df['diagnosed_diabetes']==0, 'systolic_bp'].dropna()
g1 = df.loc[df['diagnosed_diabetes']==1, 'systolic_bp'].dropna()
    
if len(g0) < 50 or len(g1) < 50:
      raise ValueError("No hay suficientes observaciones en alguno de los grupos")
    
g0_sample = g0.sample(50, replace=False)
g1_sample = g1.sample(50, replace=False)
    

# Normalidad
sh_g0 = stats.shapiro(g0_sample)
sh_g1 = stats.shapiro(g1_sample)

print(f"Normalidad No Diabetes: p = {sh_g0.pvalue:.4f} →",
      "SE CUMPLE" if sh_g0.pvalue > alpha else "NO SE CUMPLE")

print(f"Normalidad Diabetes: p = {sh_g1.pvalue:.4f} →",
      "SE CUMPLE" if sh_g1.pvalue > alpha else "NO SE CUMPLE")

# Homogeneidad de varianzas
lev = stats.levene(g0_sample, g1_sample)
print(f"Homogeneidad de varianzas (Levene): p = {lev.pvalue:.4f} →",
      "SE CUMPLE" if lev.pvalue > alpha else "NO SE CUMPLE")

Normalidad No Diabetes: p = 0.2465 → SE CUMPLE
Normalidad Diabetes: p = 0.0772 → SE CUMPLE
Homogeneidad de varianzas (Levene): p = 0.0650 → SE CUMPLE


##### Aplicación del método

In [12]:
# Nivel de significancia para la prueba
alpha = 0.05

ttest_sys = stats.ttest_ind(g1_sample, g0_sample, equal_var=False)
# La función devuelve un objeto con:
# - ttest_sys.statistic: el valor del estadístico t
# - ttest_sys.pvalue: el valor p asociado a ese estadístico

print(f"t = {ttest_sys.statistic:.3f}")
print(f"p = {ttest_sys.pvalue:.4f}")

if ttest_sys.pvalue < alpha:
    print("Conclusión estadística: Se RECHAZA la hipótesis nula.")
    print("Existe una diferencia estadísticamente significativa entre los grupos.")
else:
    print("Conclusión estadística: NO se rechaza la hipótesis nula.")
    print("No se encontraron diferencias estadísticamente significativas entre los grupos.")

t = 0.520
p = 0.6044
Conclusión estadística: NO se rechaza la hipótesis nula.
No se encontraron diferencias estadísticamente significativas entre los grupos.


**Conclusión del Análisis**
La presión sistólica promedio no difiere entre los grupos, indicando que la diabetes, por sí sola, no necesariamente se asocia con cambios en la presión arterial sistólica en la población analizada.

Esto sugiere que otros factores (edad, IMC, estilo de vida, medicación) podrían tener un impacto mayor sobre la presión arterial que el diagnóstico de diabetes por sí mismo.

Aun así, la prevención cardiovascular sigue siendo esencial para todos los pacientes, independientemente de su diagnóstico de diabetes.